# ERC-4626: vault ecosystem comparison across chains

This notebook serves both as a coding tutorial and a useful data analytics tool for ERC-4626 vaults. 

- In this notebook, we examine different ERC-4626 vaults across different EVM blockchains   
    - Currently we do not scan non-ERC-4626 vaults like Enzyme Finance, or any protocol-native vaults like Hyperliquid HPL. This is not an inherit limitation, this is not just yet implemented.
- We assemble various data tables out of the vault data to show and compare the blockchain ecosystems
- The analysis focus on USD-stablecoin nonminatd vaults
    - Currently missing are e.g. WETH vaults and staking vaults for various small cap tokens
    - There is no ERC standard for vaults fees - for some protocols we have manualled added fee reading support  
- The list of chains is somewhat randomly selected and very easy to extend to contain any chain supported by [Envio's HyperSync](https://docs.envio.dev/docs/HyperSync/hypersync-supported-networks)
- Everything is open source: You can run this notebook and associated scripts yourself on your local computer, it will take around an hour

In this notebook, we use terms Net Asset Value (NAV) and [Total Value Locked (TVL)](https://tradingstrategy.ai/glossary/total-value-locked-tvl) interchangeably.

## Usage

- Read general instructions [how to run the tutorials](./)
- See `ERC-4626 scanning all vaults onchain` example in tutorials first how to build a vault database as local `vault_db.pickle` file.




## Setup

- Set up notebook renderinb parmaeters

In [17]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

## Read scanned data

- Read the Pickle database our scanning script produced earlier 

In [18]:
import pickle
from pathlib import Path

import pandas as pd

from eth_defi.token import is_stablecoin_like

output_folder = Path("~/.tradingstrategy/vaults").expanduser()
vault_db = output_folder / "vault-db.pickle"
assert vault_db.exists(), "Run the vault scanner script first"

vault_db = pickle.load(open(vault_db, "rb"))

print(f"We have data for {len(vault_db)} vaults")

We have data for 7028 vaults


## Transform data

- Prepare the raw vault pickled data as Pandas DataFrame for data research

In [19]:
import datetime
from pprint import pformat
import pandas as pd
from eth_defi.erc_4626.hypersync_discovery import ERC4262VaultDetection
from eth_defi.chain import get_chain_name
from eth_defi.token import is_stablecoin_like

data = list(vault_db.values())
df = pd.DataFrame(data)

# print("Raw row example:")
# print(df.iloc[0])

# Build useful columns out of raw pickled Python data
# _detection_data contains entries as ERC4262VaultDetection class
entry: ERC4262VaultDetection
df["Chain"] = df["_detection_data"].apply(lambda entry: get_chain_name(entry.chain))
df["Protocol identified"] = df["_detection_data"].apply(lambda entry: entry.is_protocol_identifiable())
df["Stablecoin denominated"] = df["_denomination_token"].apply(lambda token_data: is_stablecoin_like(token_data["symbol"]) if pd.notna(token_data) else False)
df["ERC-7540"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7540())
df["ERC-7575"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7575())
df["Fee detected"] = df.apply(lambda row: (row["Mgmt fee"] is not None) or (row["Perf fee"] is not None), axis=1)
# Event counts
df["Deposit count"] = df["_detection_data"].apply(lambda entry: entry.deposit_count)
df["Redeem count"] = df["_detection_data"].apply(lambda entry: entry.redeem_count)
df["Total events"] = df["Deposit count"] + df["Redeem count"]
df["Mgmt fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Perf fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Age"] = datetime.datetime.utcnow() - df["First seen"]
df["NAV"] = df["NAV"].astype("float64")

df = df.sort_values(["Chain", "Address"])
df = df.set_index(["Chain", "Address"])

print("DataFrame MultiIndex is:", ", ".join(x for x in df.index.names))
print("DataFrame columns are:", ", ".join(x for x in df.columns))

source_df = df

display(df.sort_values("Total events", ascending=False).head())



DataFrame MultiIndex is: Chain, Address
DataFrame columns are: Symbol, Name, Denomination, NAV, Protocol, Mgmt fee, Perf fee, Shares, First seen, _detection_data, _denomination_token, _share_token, Protocol identified, Stablecoin denominated, ERC-7540, ERC-7575, Fee detected, Deposit count, Redeem count, Total events, Age


Symbol  \
Chain    Address                                                
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  yvUSDC-A   
         0xbb287e6017d3deb0e2e65061e8684eab21060123  yvUSDT-A   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72    pufETH   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1     mwETH   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688     fWETH   

                                                                      Name  \
Chain    Address                                                             
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0          USDC yVault-A   
         0xbb287e6017d3deb0e2e65061e8684eab21060123          USDT yVault-A   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72                 pufETH   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1  Moonwell Flagship ETH   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688              FARM_WETH   

                                                    Denomination        NAV  \
Chain    Address                                                              
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0         USDC 571,349.47   
         0xbb287e6017d3deb0e2e65061e8684eab21060123         USDT 462,535.14   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72         WETH  71,172.44   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1         WETH  13,597.75   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688         WETH   1,097.20   

                                                               Protocol  \
Chain    Address                                                          
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0            Yearn v3   
         0xbb287e6017d3deb0e2e65061e8684eab21060123            Yearn v3   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72  <unknown ERC-4626>   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1              Morpho   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688     Harvest Finance   

                                                      Mgmt fee   Perf fee  \
Chain    Address                                                            
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  <unknown>  <unknown>   
         0xbb287e6017d3deb0e2e65061e8684eab21060123  <unknown>  <unknown>   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72  <unknown>  <unknown>   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1       0.00       0.00   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688  <unknown>  <unknown>   

                                                                       Shares  \
Chain    Address                                                                
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0              485042.87814   
         0xbb287e6017d3deb0e2e65061e8684eab21060123             390279.176353   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72  68176.705269055443681181   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1  13470.450759909765352731   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688    922.037629422087573134   

                                                             First seen  \
Chain    Address                                                          
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0 2023-10-26 16:45:48   
         0xbb287e6017d3deb0e2e65061e8684eab21060123 2023-12-07 17:02:20   
Ethereum 0xd9a442856c234a39a81a089c06451ebaa4306a72 2024-01-31 19:44:35   
Base     0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1 2024-06-12 12:57:15   
         0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688 2023-09-07 00:27:03   

                                                                                       _detection_data  \
Chain    Address                                                                                         
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  ERC4262VaultDetection(chain=137, address='0xa0...   
         0xbb287e601

## Vaults per chain summary

- Get a summary of scanned chains at what vaults they have
- *Generic* status means that we do not have classification rules to determine the protocol on which a particular ERC-4626 vault belongs
- *Broken* status means that we could not correctly extract ERC-4626 information out of a smart contract

To detect the protocol of a vault, we need to maintain a [manual rule list here](https://github.com/tradingstrategy-ai/web3-ethereum-defi/blob/master/eth_defi/erc_4626/classification.py). Not all protocols are supported at the moment. as there are too many protocols to manually examine and identify them. Open source contributions welcome.




In [20]:
nav_threshold = 100_000
broken_max_nav_threshold = 999_000_000_000

# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
erc_7575 = df["ERC-7575"] == True 
fee_detected = df["Fee detected"] == True 

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]

# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby(level='Chain').size(),
    'Total vault TVL/NAV in stable vaults': stablecoin_only_df.groupby(level='Chain')["NAV"].sum(),
    'Protocol correctly identified': df[identified_filter].groupby(level='Chain').size().astype(int),
    'Stablecoin denominated': df[stablecoin_denominated].groupby(level='Chain').size().astype(int),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby(level='Chain').size().astype(int),
    f'ERC-7540': df[erc_7540].groupby(level='Chain').size().astype(int),
    f'ERC-7575': df[erc_7575].groupby(level='Chain').size().astype(int),
    f'Fee data supported': df[fee_detected].groupby(level='Chain').size().astype(int),
}).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the 
# NA is not present in the final results
print("Vault counts per feature per chain")
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Chain,,,,,,,,
Arbitrum,1916,"116,823,814.78",342.00,659,55.00,18.00,26.00,219.00
Avalanche,267,"2,442,252.47",26.00,58,2.00,1.00,1.00,48.00
Base,1154,"298,395,118.76",577.00,420,48.00,24.00,12.00,93.00
Berachain,232,"342,249,438.68",29.00,17,11.00,0.00,0.00,2.00
Binance,316,"220,689,980.43",18.00,84,9.00,0.00,0.00,25.00
Ethereum,2116,"9,329,184,923.52",526.00,701,177.00,16.00,14.00,177.00
Hyperliquid,12,"105,947.25",0.00,6,1.00,1.00,0.00,1.00
Mantle,37,340.36,0.00,11,0.00,0.00,0.00,5.00
Mode,56,"449,480.94",2.00,26,2.00,0.00,0.00,0.00


# Vaults per protocol summary

- Break down by identified protocol
- Because of the brokeness of EVM, Solidity and smart contract development practices, protocol-rules are hand-maintained heurestics and there can false positives and negatives

In [21]:
df = source_df.copy()
df = df.reset_index()
# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
erc_7575 = df["ERC-7575"] == True 
fee_detected = df["Fee detected"] == True 

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]
# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby('Protocol').size(),
    'Total vault TVL/NAV in stable vaults': stablecoin_only_df.groupby('Protocol')["NAV"].sum(),
    'Protocol correctly identified': df[identified_filter].groupby('Protocol').size().astype(int),
    'Stablecoin denominated': df[stablecoin_denominated].groupby('Protocol').size().astype(int),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby('Protocol').size().astype(int),
    f'ERC-7540': df[erc_7540].groupby('Protocol').size().astype(int),
    f'ERC-7575': df[erc_7575].groupby('Protocol').size().astype(int),
    f'Fee data supported': df[fee_detected].groupby('Protocol').size().astype(int),
}).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the 
# NA is not present in the final results
print("Vault counts per feature per chain")
summary_df = summary_df.sort_values("Total vault TVL/NAV in stable vaults", ascending=False)
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Protocol,,,,,,,,
<unknown ERC-4626>,4955,"8,241,114,486.62",0.00,"1,720.00",178.00,0.00,12.00,0.00
Morpho,183,"1,371,618,961.13",183.00,93.00,56.00,0.00,0.00,183.00
Fluid,57,"488,225,375.35",57.00,25.00,7.00,0.00,0.00,0.00
Euler Vault Kit,156,"91,981,535.71",156.00,33.00,10.00,0.00,0.00,0.00
Yearn v3,124,"65,471,860.68",124.00,62.00,18.00,0.00,0.00,0.00
Superform,104,"31,816,480.84",104.00,38.00,8.00,11.00,0.00,0.00
Euler,98,"28,742,970.56",98.00,28.00,6.00,0.00,0.00,0.00
Kiln Metavault,68,"19,261,194.83",68.00,60.00,8.00,0.00,0.00,0.00
Yearn compounder,71,"14,107,076.46",71.00,12.00,1.00,0.00,0.00,0.00


## Vault deployment history

- Show how much history we have for each chain


In [22]:
# Assuming your DataFrame is named 'df'
df = source_df
seen_df = df.groupby(level='Chain')['First seen'].agg(['min', 'max']).reset_index()

# Rename columns for clarity
seen_df.columns = ['Chain', 'First vault deployed', 'Last vault deployed']

seen_df = seen_df.set_index("Chain")

display(seen_df)

,First vault deployed,Last vault deployed
Chain,,
Arbitrum,2022-03-28 09:03:11,2025-04-03 20:38:18
Avalanche,2022-04-11 00:05:24,2025-04-08 19:04:39
Base,2023-08-04 15:36:07,2025-04-08 10:33:43
Berachain,2025-01-26 00:06:03,2025-04-09 07:50:02
Binance,2022-05-27 17:25:18,2025-04-08 11:29:07
Ethereum,2019-06-11 06:17:19,2025-04-07 10:13:47
Hyperliquid,2025-02-19 16:29:00,2025-04-07 15:02:00
Mantle,2023-08-22 13:25:16,2024-11-29 08:55:34
Mode,2024-03-06 23:58:43,2025-01-03 19:23:51


## Largest USD vaults

- Show the stablecoin-denominated vaults across different chains that have largest USD treasury 

In [23]:
largest_threshold = 20
largest_df = df.reset_index()
# Filter out crap
largest_df = largest_df[largest_df["Total events"] > 100] 
largest_df = largest_df[largest_df["Stablecoin denominated"] == True] 
largest_df = largest_df.sort_values(["NAV"], ascending=False)

largest_df = largest_df[["NAV", "Chain", "Address", "Name", "Denomination", "Total events", "Mgmt fee", "Perf fee"]]
largest_df = largest_df.set_index("Name")


display(largest_df.head(largest_threshold))


,NAV,Chain,Address,Denomination,Total events,Mgmt fee,Perf fee
Name,,,,,,,
Savings USDS,"3,048,906,703.48",Ethereum,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,USDS,27915,<unknown>,<unknown>
Staked USDe,"2,171,370,828.65",Ethereum,0x9d39a5de30e57443bff2a8307a4256c8797a3497,USDe,69176,<unknown>,<unknown>
Ethereal Pre-deposit Vault,"894,290,304.95",Ethereum,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,USDe,48503,<unknown>,<unknown>
Savings Dai,"518,245,450.12",Ethereum,0x83f20f44975d03b1b09e64809b757c47f942beea,DAI,72487,<unknown>,<unknown>
Bridged USDC (Stargate)Vault,"328,691,842.17",Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,129831,<unknown>,<unknown>
Usual Boosted USDC,"218,069,710.22",Ethereum,0xd63070114470f685b75b74d60eec7c1113d33a3d,USDC,17386,0.00,0.00
Fluid USD Coin,"212,698,935.23",Ethereum,0x9fb7b4477576fe5b32be4c1843afb1e55f251b33,USDC,17501,<unknown>,<unknown>
Staked USDX,"199,647,843.24",Binance,0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92,USDX,5447,<unknown>,<unknown>
Fluid Tether USD,"195,409,230.14",Ethereum,0x5c20b550819128074fd538edf79791733ccedd18,USDT,10364,<unknown>,<unknown>


## Largest USD vault per chain

- Get the largest vault of each chain

In [24]:
# Get the index of max NAV for each chain
largest_df = largest_df.reset_index().set_index(["Chain", "Name"])
max_nav_idx = largest_df.groupby('Chain')['NAV'].idxmax()
# Use these indices to get the full rows
result = largest_df.loc[max_nav_idx]

display(result)

,,NAV,Address,Denomination,Total events,Mgmt fee,Perf fee
Chain,Name,,,,,,
Arbitrum,Fluid USD Coin,"28,708,805.90",0x1a996cb54bb95462040408c06122d45d6cdb6096,USDC,33566,<unknown>,<unknown>
Avalanche,HiYield Treasury Bill Vault,"2,275,179.86",0x8475509d391e6ee5a8b7133221ce17019d307b3e,USDC,177,<unknown>,<unknown>
Base,Seamless USDC Vault,"51,330,925.98",0x616a4e1db48e22028f6bbf20444cd3b8e3273738,USDC,76596,0.00,0.00
Berachain,Bridged USDC (Stargate)Vault,"328,691,842.17",0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,129831,<unknown>,<unknown>
Binance,Staked USDX,"199,647,843.24",0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92,USDX,5447,<unknown>,<unknown>
Ethereum,Savings USDS,"3,048,906,703.48",0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,USDS,27915,<unknown>,<unknown>
Mode,USDC Ironclad Vault,"337,647.27",0x882fd369341fc435ad5e54e91d1ebc23b1fc6d4c,USDC,183,<unknown>,<unknown>
Polygon,Compound USDC,"24,765,279.22",0x781fb7f6d845e3be129289833b04d43aa8558c42,USDC,4143,0.00,0.00


## Most active vaults across all chains

- Determine vault activity by number of deposit and redeem events
- Based on all-time event count, not recent event count 
- Events may be driven by bots, so this may not reflect the popularity of a vault amount users


In [25]:
largest_threshold = 20
largest_df = df.reset_index().sort_values(["Total events"], ascending=False)

largest_df = largest_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

largest_df = largest_df.set_index("Name")

display(largest_df.head(largest_threshold))

,Total events,Chain,Address,Denomination,NAV,Age,Deposit count,Redeem count
Name,,,,,,,,
USDC yVault-A,490168,Polygon,0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0,USDC,"571,349.47",530 days 17:27:13.309733,465206,24962
USDT yVault-A,458203,Polygon,0xbb287e6017d3deb0e2e65061e8684eab21060123,USDT,"462,535.14",488 days 17:10:41.309733,407690,50513
pufETH,167837,Ethereum,0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"71,172.44",433 days 14:28:26.309733,151553,16284
Moonwell Flagship ETH,140543,Base,0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1,WETH,"13,597.75",300 days 21:15:46.309733,85385,55158
FARM_WETH,134053,Base,0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688,WETH,"1,097.20",580 days 09:45:58.309733,76070,57983
Bridged USDC (Stargate)Vault,129831,Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,"328,691,842.17",73 days 10:06:58.309733,80489,49342
Staked USDA,121756,Arbitrum,0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"913,463.57",454 days 01:40:36.309733,119741,2015
Moonwell Flagship USDC,92159,Base,0xc1256ae5ff1cf2719d4937adb3bbccab2e00a2ca,USDC,"30,150,546.03",300 days 21:15:46.309733,53220,38939
Beraborrow iBGT,90138,Berachain,0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"130,265.69",32 days 22:07:42.309733,48683,41455


## Most historically active vault per chain

- Vaults with most deposit and redeem events

In [26]:
most_active_df = df.reset_index()

most_active_df = most_active_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

# Force thousand separator
most_active_df["Total events"] = most_active_df["Total events"].astype("float64")

max_nav_idx = most_active_df.groupby('Chain')['Total events'].idxmax()
# Use these indices to get the full rows
result = most_active_df.loc[max_nav_idx]

result = result.set_index(["Chain", "Name"])

display(result)

,,Total events,Address,Denomination,NAV,Age,Deposit count,Redeem count
Chain,Name,,,,,,,
Arbitrum,Staked USDA,"121,756.00",0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"913,463.57",454 days 01:40:36.309733,119741,2015
Avalanche,GoGoPool Liquid Staking Token,"66,841.00",0xa25eaf2906fa1a3a13edac9b9657108af7b703e3,WAVAX,"848,753.39",734 days 13:02:16.309733,41381,25460
Base,Moonwell Flagship ETH,"140,543.00",0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1,WETH,"13,597.75",300 days 21:15:46.309733,85385,55158
Berachain,Bridged USDC (Stargate)Vault,"129,831.00",0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,"328,691,842.17",73 days 10:06:58.309733,80489,49342
Binance,kUSDT,"42,799.00",0x1c3f35f7883fc4ea8c4bca1507144dc6087ad0fb,VUSD,"2,623,386.84",667 days 03:04:18.309733,25329,17470
Ethereum,pufETH,"167,837.00",0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"71,172.44",433 days 14:28:26.309733,151553,16284
Hyperliquid,wHYPE,595.00,0x2831775cb5e64b1d892853893858a261e898fbeb,WHYPE,"218,912.70",27 days 12:02:01.309733,458,137
Mantle,Karak - mETH,"13,025.00",0x8529019503c5bd707d8eb98c5c87bf5237f89135,mETH,489.07,350 days 16:00:55.309733,8038,4987
Mode,Renzo aggregator,"21,420.00",0xd60dd6981ec336fda40820f8ca5e99cd17dd25a0,WETH,213.22,398 days 10:14:18.309733,12188,9232


## Oldest vaults

- Show oldest vaults

In [27]:
threshold = 1_000

oldest_df = df.reset_index()

oldest_df = oldest_df[["Chain", "Address", "Name", "Age", "Denomination", "NAV", "Total events"]]

# Force thousand separator
oldest_df["Total events"] = oldest_df["Total events"].astype("float64")

# Force event threshold to filter out some crap
oldest_df = oldest_df[oldest_df["Total events"] >= threshold]

max_nav_idx = oldest_df.groupby('Chain')['Age'].idxmax()
# Use these indices to get the full rows
result = oldest_df.loc[max_nav_idx]

result = result.set_index("Chain")

display(result)

,Address,Name,Age,Denomination,NAV,Total events
Chain,,,,,,
Arbitrum,0xf46ce0c13577232d5f29d9bd78a9cab278755346,Jones ETH,1047 days 18:57:55.309733,WETH,18.00,"2,335.00"
Avalanche,0x9dd17f32fc8355ae37425f475a10cc7bec8ca36a,,1091 days 17:15:39.309733,,0.00,"1,425.00"
Base,0xc7548d8d7560f6679e369d0556c44fe1eddea3e9,FARM_WETH,586 days 18:19:08.309733,WETH,0.97,"1,117.00"
Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,Bridged USDC (Stargate)Vault,73 days 10:06:58.309733,USDC.e,"328,691,842.17","129,831.00"
Binance,0x0f8754b36a767c5579178bd8a04d2fcd9d530b70,ygNRCH,1016 days 13:20:22.309733,NRCH,"1,188,908.73","1,195.00"
Ethereum,0x815c23eca83261b6ec689b60cc4a58b54bc24d8d,vTHOR,1079 days 22:47:24.309733,THOR,"79,463,446.44","19,573.00"
Mantle,0x8529019503c5bd707d8eb98c5c87bf5237f89135,Karak - mETH,350 days 16:00:55.309733,mETH,489.07,"13,025.00"
Mode,0xd60dd6981ec336fda40820f8ca5e99cd17dd25a0,Renzo aggregator,398 days 10:14:18.309733,WETH,213.22,"21,420.00"
Polygon,0x73958d46b7aa2bc94926d8a215fa560a5cdca3ea,Wrapped Aave Polygon GHST,1071 days 06:26:12.309733,aPolGHST,"1,145,206.25","14,511.00"


## All vaults

- List of all stablecoin vaults

In [29]:
all_df = df.reset_index()

min_nav = 10_000

stablecoin_denominated = all_df["Stablecoin denominated"] == True
all_df = all_df[stablecoin_denominated]
all_df = all_df[all_df["NAV"] < broken_max_nav_threshold]
all_df = all_df[all_df["NAV"] >= min_nav]
all_df = all_df[["NAV", "Chain", "Protocol", "Address", "Name", "Age", "Denomination", "Total events"]]
all_df = all_df.set_index("Name").sort_values("NAV", ascending=False)

print(f"Displaying {len(all_df)} stablecoin vaults w/min NAV of {min_nav} USD")
pd.options.display.max_rows = None
display(all_df)


Displaying 490 stablecoin vaults w/min NAV of 10000 USD


,NAV,Chain,Protocol,Address,Age,Denomination,Total events
Name,,,,,,,
Savings USDS,"3,048,906,703.48",Ethereum,<unknown ERC-4626>,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,203 days 20:36:14.309733,USDS,27915
Staked USDe,"2,171,370,828.65",Ethereum,<unknown ERC-4626>,0x9d39a5de30e57443bff2a8307a4256c8797a3497,505 days 02:06:50.309733,USDe,69176
Ethereal Pre-deposit Vault,"894,290,304.95",Ethereum,<unknown ERC-4626>,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,54 days 18:58:50.309733,USDe,48503
Savings Dai,"518,245,450.12",Ethereum,<unknown ERC-4626>,0x83f20f44975d03b1b09e64809b757c47f942beea,741 days 23:48:14.309733,DAI,72487
Bridged USDC (Stargate)Vault,"328,691,842.17",Berachain,<unknown ERC-4626>,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,73 days 10:06:58.309733,USDC.e,129831
Spark DAI Vault,"325,072,740.20",Ethereum,Morpho,0x73e65dbd630f90604062f6e02fab9138e713edd9,375 days 19:53:14.309733,DAI,96
Usual Boosted USDC,"218,069,710.22",Ethereum,Morpho,0xd63070114470f685b75b74d60eec7c1113d33a3d,257 days 15:18:50.309733,USDC,17386
Fluid USD Coin,"212,698,935.23",Ethereum,Fluid,0x9fb7b4477576fe5b32be4c1843afb1e55f251b33,415 days 08:57:38.309733,USDC,17501
Staked USDX,"199,647,843.24",Binance,<unknown ERC-4626>,0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92,386 days 22:34:12.309733,USDX,5447
